In [3]:
import pandas as pd
import os
from rxnmapper import RXNMapper
import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem

numexpr.utils: INFO     Note: detected 256 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
numexpr.utils: INFO     Note: NumExpr detected 256 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [2]:
path = '/home/ruard/Documents/datasets/DA_reaxys_export/'
dfs = []
for name in ['DA_1.tsv', 'DA_2.tsv', 'DA_3.tsv', 'DA_4.tsv']:
    dfs.append(pd.read_csv(os.path.join(path, name), sep='\t'))
    
df = pd.concat(dfs)
print(len(df))

df.columns

29261


Index(['Reaction ID', 'Reaction: Links to Reaxys', 'Data Count',
       'Number of Reaction Details', 'Reaction Rank', 'Record Type',
       'Reactant', 'Product', 'Bin', 'Reaction',
       'Reaction Details: Reaction Classification', 'Example label',
       'Example title', 'Fulltext of reaction', 'Number of Reaction Steps',
       'Multi-step Scheme', 'Multi-step Details', 'Number of Stages',
       'Solid Phase', 'Time (Reaction Details) [h]',
       'Temperature (Reaction Details) [C]',
       'Pressure (Reaction Details) [Torr]', 'pH-Value (Reaction Details)',
       'Other Conditions', 'Reaction Type', 'Subject Studied',
       'Prototype Reaction', 'Named Reaction',
       'Type of reaction description (Reaction Details)', 'Location',
       'Comment (Reaction Details)', 'Product.1', 'Yield', 'Yield (numerical)',
       'Yield (optical)', 'Stage Reactant', 'Reagent', 'Catalyst',
       'Solvent (Reaction Details)', 'References', 'Links to Reaxys',
       'Unnamed: 41'],
      dt

In [3]:
da_rxn_smarts = AllChem.ReactionFromSmarts(
    '[#6:1]=[#6:2].[#6:3]=[#6:4][#6:5]=[#6:6]>>[#6:1]1[#6:2][#6:3][#6:4]=[#6:5][#6:6]1'
)

# diene_smarts = Chem.MolFromSmarts('[C,c,N,n,O]=[C,c,N,n][C,c,N,n]=[C,c,N,n,O]')
# dienophile_smarts = Chem.MolFromSmarts('[C,c,N,n]=[C,c,N,n]')
diene_smarts = Chem.MolFromSmarts('[C,c]=[C,c][C,c]=[C,c]')
dienophile_smarts = Chem.MolFromSmarts('[C,c]=[C,c]')

def simulate_da_reaction(substrates):
    products = []
    products += da_rxn_smarts.RunReactants(substrates)
    substrates = [substrates[1], substrates[0]]
    products += da_rxn_smarts.RunReactants(substrates)
    
    products = [Chem.MolToSmiles(product[0]) for product in products]
    products = list(set(products))
    return [Chem.MolFromSmiles(product) for product in products]

products = simulate_da_reaction([Chem.MolFromSmiles(smi) for smi in ["C=C", "C=CC=C"]])
Chem.MolToSmiles(products[0])

'C1=CCCCC1'

In [4]:
# no multistep reactions
df = df[df['Multi-step Details'].isnull()]
print(len(df))

# no NaN reactions 
df = df[~df['Reaction'].isna()]
print(len(df))

# only bimolecular reactions involving dienes and dienophiles 
filtered_reaction_ids = []
for reaction, reaction_id in zip(df['Reaction'].values, df['Reaction ID'].values):
    reactants, products = reaction.split('>>')
    if len(reactants.split('.')) == 2 and len(products.split('.')) == 1: 
        reactant1, reactant2 = reactants.split('.')
        reactant1 = Chem.MolFromSmiles(reactant1)
        reactant2 = Chem.MolFromSmiles(reactant2)
        product = Chem.MolFromSmiles(products)
        
        if None not in [reactant1, reactant2, product]:
            da_products = simulate_da_reaction([reactant1, reactant2])
            if len(da_products) > 0: 
                filtered_reaction_ids.append(reaction_id)
        
df = df[df['Reaction ID'].isin(filtered_reaction_ids)]
print(len(df))

# no duplicate reaction smiles
df = df.drop_duplicates(subset=['Reaction'])
print(len(df))

29241
28349
6519
4467


In [5]:
df.to_csv('/home/ruard/Documents/datasets/DA_reaxys_export/DA_raw.csv')

## Regioselectivity filter

In [1]:
df = pd.read_csv('/home/ruard/Documents/datasets/DA_reaxys_export/DA_raw.csv')

NameError: name 'pd' is not defined

In [2]:
from autode.solvent.solvents import solvents
SOLVENTS = solvents

def normalize_mol(mol):
    return Chem.MolFromSmiles(Chem.MolToSmiles(mol))

def get_mols_from_reaction_smiles(reaction_smiles):
    reactants, product = reaction_smiles.split('>>')
    reactant1, reactant2 = reactants.split('.')
    reactant1 = Chem.MolFromSmiles(reactant1)
    reactant2 = Chem.MolFromSmiles(reactant2)
    product = Chem.MolFromSmiles(product)
    
    return (reactant1, reactant2), product


def check_if_solvent_available(solvent_string, program):
    # NaN solvent
    if type(solvent_string) is float:
        return False
    
    # parse list of provided solvents
    if ';' in solvent_string:
        solvent_strings = solvent_string.split(';')
    else:
        solvent_strings = [solvent_string]
    
    # check for each solvent if they're available
    for string in solvent_strings:
        for solvent in SOLVENTS:
            if solvent_string in solvent.aliases:
                if hasattr(solvent, program):
                    return True
    
    return False


def get_solvent(solvent_string, program):
    for solvent in SOLVENTS:
        if solvent_string in solvent.aliases:
            return getattr(solvent, program)

rdkit       : INFO     Enabling RDKit 2022.09.5 jupyter extensions


In [8]:
# first let's remove stereochemistry to avoid having stereoisomer duplicates
reaction_smiles_nostereo = []
for reaction_smiles in df['Reaction'].values:
    reactants, product = get_mols_from_reaction_smiles(reaction_smiles)
    for reactant in reactants:
        Chem.RemoveStereochemistry(reactant)
    Chem.RemoveStereochemistry(product)
    reaction_smiles_nostereo.append(
        f'{Chem.MolToSmiles(reactants[0])}.{Chem.MolToSmiles(reactants[1])}>>{Chem.MolToSmiles(product)}'
    )
    
df['Reaction'] = reaction_smiles_nostereo

print(len(df))
df = df.drop_duplicates(subset=['Reaction'])
print(len(df))

4467
4175


In [9]:
# now we have to figure out if both motifs are symmetrically substituted
filtered_reaction_ids = []
for reaction_smiles, reaction_id in zip(df['Reaction'].values, df['Reaction ID'].values):
    reactants, product = get_mols_from_reaction_smiles(reaction_smiles)
    da_products = simulate_da_reaction(reactants)
    if len(da_products) >= 2: 
        filtered_reaction_ids.append(reaction_id)
        
df = df[df['Reaction ID'].isin(filtered_reaction_ids)]
print(len(df))

1691


In [10]:
# # filter reactions on their yield
# filtered_reaction_ids = []
# for chem_yield, reaction_id in zip(df['Yield (numerical)'].values, df['Reaction ID'].values):
#     if str(chem_yield) != 'nan':
#         try:
#             chem_yield = float(chem_yield)
#         except:
#             chem_yield = float(chem_yield.split(';')[0])

#         if chem_yield >= 50:
#             filtered_reaction_ids.append(reaction_id)
        
# df = df[df['Reaction ID'].isin(filtered_reaction_ids)]
# print(len(df))

In [12]:
# create Dataset here
max_n_heavy_atoms = 25
program = 'orca'
add_solvent = True
save_path = "/home/ruard/code/virtual_reactions/data/datasets/da//DA_regio_orca_solvent.csv"


reaction_idx = []
substrates = []
products = []
solvents = []
reaction_smiles_list = []
labels = []

for idx, row in df.iterrows():
    solvent = row['Solvent (Reaction Details)']
    reactants, product = get_mols_from_reaction_smiles(row['Reaction'])
    da_products = simulate_da_reaction(reactants)
    
    product_smiles = Chem.MolToSmiles(normalize_mol(product))
    da_products = [Chem.MolToSmiles(normalize_mol(mol)) for mol in da_products]
    
    matches = sum([mol == product_smiles for mol in da_products])
    
    # check 1) "real" product mol is among simulated products
    #       2)  product is not larger than specified size
    #       3) solvent is available
    if matches == 1 and product.GetNumHeavyAtoms() < max_n_heavy_atoms:
        if (add_solvent and check_if_solvent_available(solvent, program)) or (not add_solvent):
            
            # TODO: remove this - manually filter out site-selective one
            if len(da_products) <= 4:
                for mol in da_products:
                    reactant1_smiles, reactant2_smiles = row['Reaction'].split('>>')[0].split('.')
                    reaction_smiles = f"{reactant1_smiles}.{reactant2_smiles}>>{mol}"
                    substrates.append(f"{reactant1_smiles}.{reactant2_smiles}")
                    products.append(mol)

                    if add_solvent:
                        solvents.append(get_solvent(solvent, program))
                    else:
                        solvents.append(None)

                    reaction_smiles_list.append(reaction_smiles)
                    reaction_idx.append(idx)
                    if mol == product_smiles:
                        labels.append(1)
                    else:
                        labels.append(0)   
    
print(len(reaction_idx), len(set(reaction_idx)))    
    
dataset = pd.DataFrame({
    'reaction_idx': reaction_idx,
    'uid': np.arange(len(reaction_idx)),
    'substrates': substrates,
    'products': products,
    'solvent': solvents,
    'reaction_smiles': reaction_smiles_list,
    'label': labels,
    'simulation_idx': np.zeros(len(reaction_idx))
})   
dataset
    
dataset.to_csv(save_path)

735 340


In [38]:
amine_edg_smarts = Chem.MolFromSmarts('C1=CC(C2)CCC12')

i = 0
products = dataset['products'].values
products = [Chem.MolFromSmiles(p) for p in products]
for idx, p in enumerate(products):
    if len(p.GetSubstructMatches(amine_edg_smarts)) >= 1:
        i += 1

print(i, len(products))

104 735
